In [ ]:
import os
import ipyvuetify as v
from astropy.utils.data import download_file
import ipywidgets as widgets
import traitlets
from jdaviz.app import Application
from jdaviz.core.config import get_configuration
from jdaviz.core.data_formats import get_valid_format
#from specutils.io.registers import identify_spectrum_format
from jdaviz import SpecViz, CubeViz

In [ ]:
# app = Application(configuration='default')
# app

In [ ]:
# local = '/Users/bcherinka/Work/notebooks/datb/manga-7495-12704-LOGCUBE.fits'
# remote = download_file('https://data.sdss.org/sas/dr15/manga/spectro/redux/v2_4_3/8485/stack/manga-8485-1901-LOGCUBE.fits.gz', cache=True)
# jwstlocal = '/Users/bcherinka/Downloads/jw00626-o030_s00000_nirspec_f170lp-g235m_x1d.fits'

In [ ]:
# this is a widget for debugging
out = widgets.Output()
out.add_traits(_metadata=traitlets.Dict(default_value={'mount_id': 'out'}).tag(sync=True))
out

In [ ]:
# set the jwst target sources and randomly select one to start with
jwst_sources = [227, 482, 546, 1186]
#selected_src = jwst_sources[np.random.randint(0,3)]

@out.capture()
def prep_data(source, stype=None):
    ''' prepare local test data '''
    
    #path = os.getenv("JWST_VOILA_TEST_DATA", os.path.abspath(os.curdir))
    #path1d = os.path.join(path, 'jwst_level3_NIRSpec/jw95065_{0}_nrs_msaspec_barshadow_x1d.fits'.format(source))
    #path2d = os.path.join(path, 'jwst_level3_NIRSpec/jw95065_{0}_nrs_msaspec_barshadow_s2d.fits'.format(source))
    #pathimg = os.path.join(path, 'jwst_level3_NIRSpec/cutouts/SOURCEID_{0}.fits'.format(source))    

    path = os.getenv("JWST_VOILA_SPECVIZ_DATA", os.path.abspath(os.curdir))
    path1d = os.path.join(path,'jw00626-o030_s00000_nirspec_f170lp-g235m_x1d.fits')
    path2d = os.path.join(path, 'jw00626-o030_s00000_nirspec_f170lp-g235m_s2d.fits')
    path3d = os.path.join(path, 'single_nrs1-nrs2_g395h-f290lp_s3d.fits')

    #print('local data path', path)
    #print('local 1d path', path1d)
    #print('local img path', pathimg)
    
    if stype == '3d':
        return path3d
    elif stype == '1d':
        return path1d
    elif stype == '2d':
        return path2d
    else:
        return path1d

In [ ]:
def get_file_from_target_name(name):
    ''' this function looks up the filename based on a target id/coord, etc 
    
    If the query parameter sent from the Portral is a target name or coordinate, we need to look it up 
    using Astroquery and retrieve the relevant datafile URI.  If the query parameter is a datafile URI, then we can 
    pass that directly to JDAViz.
    
    This function should be moved into the ASB back-end. It should determine the data URI, pass it back to the
    front-end so Vue can send it this notebook.  The notebook (or the Application), should contain the logic
    to parse the data URL and determine the configuration and default data to load.
    
    Pseudo-code:
    if name.endswith('.fits'):
        return name
    else:
        return get_dataURI_from_astroquery(name)
    
    This is a toy hack to "get" the filename, or data URI.  Everything is local.  
    
    Changing this hack to use a data uri
    '''
#     if name == '3d':
#        # fake target id for a data cube
#        datafile = prep_data(name, stype='3d')
#     elif name == '2d':
#         datafile = prep_data(name, stype='2d')
#     elif name == '1d':
#         datafile = prep_data(name, stype='1d')
#     else:
#         datafile = prep_data(name, stype='1d')
    datafile = prep_data(name, stype=name)
    return datafile

In [ ]:
def determine_input_from_file(path):
    ''' this function determines the input data based on filename
    
    It should determine the type of application configuration to use and which data extension to load
    by default. 
    
    '''

    import specutils.io.default_loaders.jwst_reader as jw

    # custom mast configuration
    mastconfig = {'settings': {'configuration': 'mastviz', 'viewer_spec': '',
      'visible': {'menu_bar': False,
       'toolbar': False,
       'tray': False,
       'tab_headers': False},
      'context': {'notebook': {'max_height': '600px'}}},
     'toolbar': ['g-subset-tools', 'g-unified-slider'],
     'tray': ['g-gaussian-smooth', 'g-collapse'],
     'viewer_area': []}    
    
    
    #if path.endswith('_s3d.fits'):
    if jw.identify_jwst_s3d_fits(None, path):
        #return 'cubeviz'
        varea = get_configuration('cubeviz', section='viewer_area')
        mastconfig['settings']['viewer_spec']='cubeviz'
    elif jw.identify_jwst_x1d_fits(None, path):
    #elif path.endswith(('_x1d.fits','_x1dints.fits','_c1d.fits')):
        #return 'specviz'
        varea = get_configuration('specviz', section='viewer_area')
        mastconfig['settings']['viewer_spec']='specviz'
    #elif path.endswith('_s2d.fits'):
    elif jw.identify_jwst_s2d_fits(None, path):
        # custom viewer configuration
        viewers = [{'name': 'Image', 'plot': 'g-image-viewer', 'reference': 'image-viewer'}]
        varea = [{'container': 'col', 'children': [{'container': 'row', 'viewers': viewers}]}]
        mastconfig['settings']['viewer_spec']='imviz'
        #mastconfig['viewer_area'] = viewer_area
        #return mastconfig
    else:
        # return 'default'
        varea = []
        mastconfig['settings']['viewer_spec']='default'
    
    #print('mastconfig', mastconfig)
    #print('varea', varea)
    mastconfig['viewer_area'] = varea
    #print('cfg', mastconfig)
    return mastconfig


In [ ]:
default_file_to_config_mapping = {'JWST x1d': 'specviz', 'JWST s2d': 'imviz', 'JWST s3d': 'cubeviz', 
                                  'MaNGA cube': 'cubeviz'}

In [ ]:
def determine_configuration(path):
    valid_format, config = get_valid_format(path)
#     valid_format = identify_spectrum_format(path)
#     if valid_format:
#         valid_format = valid_format[0] if len(valid_format) > 1 else valid_format
#         config = default_file_to_config_mapping.get(valid_format, None)

    # custom mast configuration    
    mastconfig = {
        'settings': {
            'configuration': 'mastviz', 'viewer_spec': '',
            'visible': {'menu_bar': False, 
                        'toolbar': False,
                        'tray': False,
                        'tab_headers': False},
            'context': {'notebook': {'max_height': '600px'}}
        },
        'toolbar': ['g-subset-tools'],
        'tray': ['g-gaussian-smooth'],
        'viewer_area': []
    }
    
    if valid_format == 'JWST s3d':
        # access cubeviz config
        cube_config = get_configuration('cubeviz')
        # re-add the data parser settings
        data = cube_config['settings']['data']
        mastconfig['settings']['data'] = data
        # re-add the cube toolbar but remove the g-data-tools
        tools = cube_config['toolbar']
        tools.remove('g-data-tools')
        mastconfig['toolbar'] = tools
        # re-add the tray 
        mastconfig['tray'] = cube_config['tray']
        # get the viewer area for cubeviz
        varea = cube_config['viewer_area']
        mastconfig['settings']['viewer_spec']='cubeviz'
    elif valid_format == 'JWST x1d':
        # get the viewer area for specviz
        spec_config = get_configuration('specviz')
        varea = spec_config['viewer_area']
        mastconfig['settings']['viewer_spec']='specviz'
        mastconfig['tray'] = spec_config['tray']
    elif valid_format == 'JWST s2d':
        # define a custom viewer area
        viewers = [{'name': 'Image', 'plot': 'g-image-viewer', 'reference': 'image-viewer'}]
        varea = [{'container': 'col', 'children': [{'container': 'row', 'viewers': viewers}]}]
        mastconfig['settings']['viewer_spec']='imviz'
    else:
        varea = []
        mastconfig['settings']['viewer_spec']='default'

    mastconfig['viewer_area'] = varea
    return mastconfig

In [ ]:
def load_default_data(app, config, datafile):
    ''' this should load default data based on configuration and input datafile '''
    if config['settings'].get('viewer_spec', '') == 'cubeviz':
        #app.add_data_to_viewer('flux-viewer', datafile, ext='SCI')
        #app.add_data_to_viewer('uncert-viewer', datafile, ext='ERR')
        #app.add_data_to_viewer('mask-viewer', datafile, ext='DQ')
        #app.add_data_to_viewer('spectrum-viewer', datafile, ext='SCI')
        pass
    elif config['settings'].get('viewer_spec', '') == 'specviz':
        #app.add_data_to_viewer('spectrum-viewer', datafile, ext='EXTRACT1D')
        pass
    elif config['settings'].get('viewer_spec', '') == 'imviz':
        app.add_data_to_viewer('image-viewer', datafile, ext='SCI')
    else:
        pass


In [ ]:
def reconfigure_application(config):
    app.reset_configuration()
    app.load_configuration(config)


In [ ]:
def create_app(config):
    app = Application(configuration=config)
    if config['settings'].get('viewer_spec', '') == 'cubeviz':
        # replace with helper = CubeViz()
        helper = app
    elif config['settings'].get('viewer_spec', '') == 'specviz':
        helper = SpecViz(app)
        #app = helper.app
        #app.reset_configuration()
        #app.load_configuration(config)
    elif config['settings'].get('viewer_spec', '') == 'imviz':
        #app = Application(configuration=config)
        helper = app
    else:
        #app = Application(configuration=config)
        helper = app
    return app, helper

In [ ]:
@out.capture()
def update_data_handler(widget, event, data):
    # get new data from extracted data uri
    #selected_src = int(data)
    selected_src = data
    print('selected_src', selected_src)

    datafile = get_file_from_target_name(selected_src)
    #config = determine_input_from_file(datafile)
    config = determine_configuration(datafile)
    #reconfigure_application(config)
    app, helper = create_app(config)
    helper.load_data(datafile)
    load_default_data(app, config, datafile)
    col.children = [app]

    


In [ ]:
# col = v.Col(xs12=True, lg6=True, xl4=True, children=[app])
# row = v.Row(_metadata={'mount_id': 'specviz'}, dense=True, row=True, wrap=True, align_center=True, children=[col])
# row.on_event('change', update_data_handler)

In [ ]:
col = v.Col(_metadata={'mount_id': 'specviz'}, cols="9", children=["Loading..."])
#row = v.Row(_metadata={'mount_id': 'specviz'}, dense=True, row=True, wrap=True, align_center=True, children=[col])
col.on_event('load', update_data_handler)